In [3]:
import os
TOKEN = os.environ.get('HF_TOKEN', '')

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Specify the model name or path
MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"

def load_llama_model(model_name=MODEL_NAME, device='cuda', token=TOKEN):
    """
    Load the LLaMA model and tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name,use_auth_token=token)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype="auto",  # Automatically selects FP16 if GPU is used
        device_map="auto",    # Automatically maps the model to GPU
        use_auth_token=token
    )
    return model, tokenizer


In [5]:
import torch

def generate_llama_response(prompt, model, tokenizer, max_length=200, temperature=0.7, top_p=0.9):
    """
    Generate a response to a given prompt using the LLaMA model.
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            eos_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


In [6]:
def unload_llama_model(model):
    """
    Unload the model and clear GPU memory.
    """
    del model
    torch.cuda.empty_cache()


In [16]:
from textwrap import TextWrapper

def wrap_print_text(print):
    """Adapted from: https://stackoverflow.com/questions/27621655/how-to-overload-print-function-to-expand-its-functionality/27621927"""

    def wrapped_func(text):
        if not isinstance(text, str):
            text = str(text)
        wrapper = TextWrapper(
            width=80,
            break_long_words=True,
            break_on_hyphens=False,
            replace_whitespace=False,
        )
        return print("\n".join(wrapper.fill(line) for line in text.split("\n")))

    return wrapped_func

# Wrap the print function
print = wrap_print_text(print)

In [7]:
model, tokenizer = load_llama_model()

c:\Users\bagge\miniforge-pypy3\envs\DS776_env\Lib\site-packages\transformers\models\auto\tokenization_auto.py:796: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
c:\Users\bagge\miniforge-pypy3\envs\DS776_env\Lib\site-packages\transformers\models\auto\auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [12]:
def generate_response(prompt, model, tokenizer, max_length=200, temperature=0.7, top_p=0.9):
    """
    Generate a response to a given prompt using the LLaMA model.
    """
    # Ensure the tokenizer has a padding token
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,  # Ensures padding is applied if needed
        truncation=True,
        max_length=max_length,
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


In [13]:
response = generate_response("How many moons does Jupiter have?", model, tokenizer)

In [17]:
print(response)

How many moons does Jupiter have? As of my last update in 2023, Jupiter has a
total of 92 confirmed moons. The largest moon of Jupiter is Ganymede, which is
also the largest moon in the solar system.

Here's a list of the moons of Jupiter, with their sizes in kilometers (km) and
their orbital periods in days (days):

1. **Io** - 4,266 km
2. **Europa** - 4,879 km
3. **Ganymede** - 5,262 km
4. **Callisto** - 4,821 km
5. **Amalthea** - 1,433 km
6. **Thebe** - 1,457 km
7. **Leda** - 1,434 km
8. ** Himalia** - 1,394 km
9. **Elara** - 1,394 km
10. **Car


In [19]:
unload_model(model)